# 机场廊桥派工人员模拟环境

机场廊桥过程中涉及以下智能体：
1. 航班Flights，航班本身具有发车时间和到达时间的属性，由此诞生出任务的概念
2. 勤务人员 individual，机场具有的人员的资质属性、位置等，动态更新是否分配任务

在智能体下会产生不同的传输子类
1. 任务类 Task，介绍航班出港或者入港带来的状态更新
2. 航司类 AviationCompany, 介绍航司的基本信息

同时存在环境 Model
1. 环境作用：规范仿真过程中的时间和空间模块
2. 对观察的信息作为一定的输入，不同更新 modeld 的状态

In [1]:
import mesa
import pandas as pd
import numpy as np

# 机场环境类 Model

In [26]:
class Task():
    def __init__(self) -> None:
        # 任务需要有位置、任务选项
        self.location = None
        self.time = None
        self.taskList = []

class AviationCompany():
    def __init__(self):
        self.name = None
        self.codingId = None
        self.terminal = 'T2' # set for T2 or S2
        self.isYiBan = False
        self.isFangXing = False
        self.isWeiXiu = False
        self.isZhongwen = False
        self.isYingwen = False
    
    def create_task(self):
        newTask = Task()
        newTask.location = self.terminal
        newTask.taskList = [self.isYiBan,self.isFangXing,self.isWeiXiu,self.isZhongwen,self.isYingwen]
        return newTask

class ACsets():
    def __init__(self):
        self.dfCompany = pd.DataFrame(columns=['name', 'codingId', 'terminal', 'isYiBan', 'isFangXing', 'isWeiXiu', 'isZhongWen', 'isYingWen'])
        self.companySets = set()
    
    def login(self,path):
        # 根据 path 提供的航司资质来注册航司列表
        df_com = pd.read_excel(path)
        print(df_com.head())
        df_com.drop(['航班类型'], axis=1, inplace=True)
        # 重命名列
        df_com.rename(columns={'航空公司':'name', '二字代码':'codingId', '停机坪':'terminal','一般勤务':'isYiBan','放行机务':'isFangXing','维修机务':'isWeiXiu','英文耳机':'isYingWen','中文耳机':'isZhongWen'}, inplace=True)
        # 重命列
        self.dfCompany = df_com
        self.companySets = set(df_com['codingId'])
    
    def get_company(self,codingID):
        # 根据二字代码获取航司信息
        avi = AviationCompany()
        temp_row = self.dfCompany[self.dfCompany['codingId'] == codingID]
        if temp_row.empty:
            return 'The company is not in the list'
        else:
            avi.name = temp_row['name'].values[0]
            avi.codingId = temp_row['codingId'].values[0]
            avi.terminal = temp_row['terminal'].values[0]
            avi.isYiBan = temp_row['isYiBan'].values[0]
            avi.isFangXing = temp_row['isFangXing'].values[0]
            avi.isWeiXiu = temp_row['isWeiXiu'].values[0]
            avi.isZhongwen = temp_row['isZhongWen'].values[0]
            avi.isYingwen = temp_row['isYingWen'].values[0]
            return avi

path =  '../dataset/new_aviationCompany.xlsx'
acsets = ACsets()
acsets.login(path)

    航空公司 二字代码 航班类型 停机坪  一般勤务  放行机务  维修机务  英文耳机  中文耳机
0   国泰航空   CX    客  T2     1     0     0     0     0
1   美国航空   AA    客  T2     1     0     0     0     0
2  阿联酋航空   EK    客  T2     1     0     0     0     0
3  俄罗斯航空   SU    客  T2     1     0     0     1     0
4  加拿大航空   AC    客  T2     1     0     0     0     0


In [27]:
class TaskSets():
    def __init__(self):
        # 新增 task
        # 删除 task 
        # 获取 task 的统计信息
        self.dfTask = pd.DataFrame(columns=['location', 'time', 'taskList'])
        self.taskSets = set()
    
    def login(self,path):
        pass 

    def append_task(self,task):
        self.dfTask = self.dfTask.append({'location': task.location, 'time': task.time, 'taskList': task.taskList}, ignore_index=True)
        self.taskSets.add(task)

In [25]:
class Airports(mesa.Model):
    def __init__(self, num_airports, num_flights):
        # TODO 暂时未完成
        super().__init__()
        self.num_airports = num_airports
        self.num_flights = num_flights
        self.airports = pd.DataFrame(columns=['airport_id', 'x', 'y'])
        self.flights = pd.DataFrame(columns=['flight_id', 'origin', 'destination', 'departure_time', 'arrival_time'])
        self.airport_id = 0
        self.flight_id = 0
        self.create_airports()
        self.create_flights()
    
    def step(self) -> None:
        return super().step()
    
    def create_airports(self) -> None:
        for i in range(self.num_airports):
            # Create a new airport with a random x and y coordinate
            new_row = {'airport_id': self.airport_id, 'x': np.random.uniform(0, 100), 'y': np.random.uniform(0, 100)}
            self.airports = pd.concat([self.airports, pd.DataFrame(new_row, index=[0])], ignore_index=True)
            self.airport_id += 1
            
    def create_flights(self) -> None:
        for i in range(self.num_flights):
            origin = np.random.choice(self.airports['airport_id'])
            destination = np.random.choice(self.airports['airport_id'])
            while destination == origin:
                destination = np.random.choice(self.airports['airport_id'])
            departure_time = np.random.uniform(0, 24)
            arrival_time = np.random.uniform(0, 24)
            new_row = {'flight_id': self.flight_id, 'origin': origin, 'destination': destination, 'departure_time': departure_time, 'arrival_time': arrival_time}
            self.flights = pd.concat([self.flights, pd.DataFrame(new_row, index=[0])], ignore_index=True)
            self.flight_id += 1

airport = Airports(10, 100)

/tmp/ipykernel_131755/3809109187.py:21: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.airports = pd.concat([self.airports, pd.DataFrame(new_row, index=[0])], ignore_index=True)
/tmp/ipykernel_131755/3809109187.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.flights = pd.concat([self.flights, pd.DataFrame(new_row, index=[0])], ignore_index=True)


#  人员类 individual 和 机组人员类 staff

人员类需要具有的属性包括：
1. 姓名
2. 工号
3. 资质 1
4. 资质 2 
5. 资质 3
6. 资质 4
7. 组别(对应四个休息室)
8. 位置

人员类的方法
TODO 2024-03-21 待定

In [10]:
class individual():
    def __init__(self, unique_id, model):
        self.name = None
        # 增加基础属性
        self.isYiBan = False
        self.isLangQiao = False
        self.isQiaozai = False
        self.isZhongwen = False
        self.isYingyu = False
        self.workDuration = 0
        self.restPos = 183 # 183\60\70\39
        self.pos = 50 # 机位的选择
        self.isWork = False
    
    def step(self):
        print('individual step')

class staffSets(mesa.Agent):
    def __init__(self, num_individuals, airport):
        super().__init__()
        self.num_individuals = num_individuals
        self.individuals = pd.DataFrame(columns=['individual_id', 'name', 'isYiBan', 'isLangQiao', 'isQiaozai', 'isZhongwen', 'isYingyu', 'workDuration', 'restPos', 'pos', 'isWork'])
        
    
    def login(self,path):
        # 根据资质来注册员工列表
        pass

    def append_individual(self,individual):
        self.individuals.append(individual)
    
    def get_Idle_individual(self):
        # 获取空闲的员工
        pass

    
    def step(self):
        for individual in self.individuals:
            individual.step()

individual = individual(1, airport)